在鐵達尼資料集中，今天我們專注觀察變數之間的相關性，以Titanic_train.csv 中，首先將有遺失值的數值刪除，我們取 Titanic_train.csv 的年齡資料，試著將課堂中所學的方法應用上去。

Q1: 產生一個新的變數(Age_above65_) Age>=65為 'Y'，其餘為'N'。

Q2: 添加女性和男性，產生一個新的變數(Age_above65_female)，女性或Age>=65為'Y'，其餘為'N'。

Q3: 透過昨天課程的內容，驗證產生的兩個新變數，哪一個和目標變數(Survived_cate)的相關性較高?

In [1]:
import pandas as pd
import researchpy

In [2]:
df = pd.read_csv("titanic/Titanic_train.csv")
df = df[["Sex", "Age", "Survived"]]
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 3 columns):
Sex         714 non-null object
Age         714 non-null float64
Survived    714 non-null int64
dtypes: float64(1), int64(1), object(1)
memory usage: 22.3+ KB


In [3]:
def age65(x):
    if x >= 65:
        return "Y"
    else:
        return "N"
    
def agefemale(row):
    if row.Sex == "female":
        return "Y"
    else:
        if row.Age >= 65:
            return "Y"
        else:
            return "N"
        
def judgment_CramerV(df,V):
    if df == 1:
        if V < 0.10:
            qual = 'negligible'
        elif V < 0.30:
            qual = 'small'
        elif V < 0.50:
            qual = 'medium'
        else:
            qual = 'large'
    elif df == 2:
        if V < 0.07:
            qual = 'negligible'
        elif V < 0.21:
            qual = 'small'
        elif V < 0.35:
            qual = 'medium'
        else:
            qual = 'large'
    elif df == 3:
        if V < 0.06:
            qual = 'negligible'
        elif V < 0.17:
            qual = 'small'
        elif V < 0.29:
            qual = 'medium'
        else:
            qual = 'large'
    elif df == 4:
        if V < 0.05:
            qual = 'negligible'
        elif V < 0.15:
            qual = 'small'
        elif V < 0.25:
            qual = 'medium'
        else:
            qual = 'large'
    else:
        if V < 0.05:
            qual = 'negligible'
        elif V < 0.13:
            qual = 'small'
        elif V < 0.22:
            qual = 'medium'
        else:
            qual = 'large'
    return(qual)

In [4]:
#Q1 新增變數
df["Ageabove65"] = df["Age"].apply(age65)
df.head()

,Sex,Age,Survived,Ageabove65
0,male,22.0,0,N
1,female,38.0,1,N
2,female,26.0,1,N
3,female,35.0,1,N
4,male,35.0,0,N


In [5]:
# Q2 新增變數
df["Age_above65_female"] = df.apply(agefemale, axis=1)
df.head()

,Sex,Age,Survived,Ageabove65,Age_above65_female
0,male,22.0,0,N,N
1,female,38.0,1,N,Y
2,female,26.0,1,N,Y
3,female,35.0,1,N,Y
4,male,35.0,0,N,N


In [6]:
# Q3 驗證新產生的兩個新變數，哪個和 Survived 的相關性更高？
# 按照之前的範例先將 Survived 轉成類別型態
df["Survived_cate"] = df["Survived"].astype("object")
df_select = df[["Survived_cate", "Ageabove65", "Age_above65_female"]]
df_select.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 3 columns):
Survived_cate         714 non-null object
Ageabove65            714 non-null object
Age_above65_female    714 non-null object
dtypes: object(3)
memory usage: 22.3+ KB


In [7]:
# 由於都是類別型，離散的資料，我們用 Cramer V
# age>=65
conTable = pd.crosstab(df_select["Ageabove65"], df_select["Survived_cate"])
df_cramer = min(conTable.shape[0], conTable.shape[1]) - 1
crosstab, res = researchpy.crosstab(df_select["Survived_cate"], df_select["Ageabove65"],
                                   test="chi-square")
print("Cramer's value is {}".format(res.loc[2, "results"]))
print(judgment_CramerV(df_cramer, res.loc[2, "results"]))

Cramer's value is 0.0803
negligible


In [8]:
# age>-65, and female
conTable = pd.crosstab(df_select["Age_above65_female"], df_select["Survived_cate"])
df_cramer = min(conTable.shape[0], conTable.shape[1]) - 1
crosstab, res = researchpy.crosstab(df_select["Survived_cate"], 
                                    df_select["Age_above65_female"],
                                    test="chi-square")
print("Cramer's value is {}".format(res.loc[2, "results"]))
print(judgment_CramerV(df_cramer, res.loc[2, "results"]))

Cramer's value is 0.514
large
